In [1]:
import pandas as pd
import json, os

"""
多少站
多少桩
功率
采样率
"""

'\n多少站\n多少桩\n功率\n采样率\n'

In [4]:
countries = pd.read_csv("Data2025\\CountryList.csv", dtype=str)
countries.fillna({"ISO": "NA"}, inplace=True)

In [6]:
result = []
for country in countries["ISO"].to_list():
    path = os.path.join("Data2025", country + "Charger")
    with open(path + ".json", 'r') as j:
        data = json.load(j)
    chargingNum = 0
    power = set()
    fastStation = 0
    fastCharging = 0
    for i in data:
        NoP = 0 if i["NumberOfPoints"] is None else i["NumberOfPoints"]
        Q = 0
        fast = False
        if i["Connections"] == []:
            Q += 1
        else:
            for j in i["Connections"]:
                quantity = j["Quantity"]
                powerKW = j["PowerKW"]
                if quantity is None:
                    Q += 1
                else:
                    Q += quantity
                power.add(powerKW)
                if powerKW is not None and powerKW >= 20:
                    fastCharging += 1
                    if not fast:
                        fastStation += 1
                        fast = True
        chargingNum += max(NoP, Q)
    
    index = countries[countries["ISO"] == country]
    SiW = index["Location Number"].tolist()[0] if not index.empty else None
    CiW = index["Station Number"].tolist()[0] if not index.empty else None
    d = {
        "Country": country,
        "StationNum": len(data),
        "StationNum in Web": SiW,
        "ChargingNum": chargingNum,
        "ChargingNum in Web": CiW,
        "Power": power,
        "Fast Station Num (Contain at least one fast charging)": fastStation,
        "Fast Charging Num (Power >= 20kW)": fastCharging,
    }
    result.append(d)
pd.DataFrame(result).to_csv("statistic.csv", encoding="utf-8")

Summary cleanded data

In [37]:
import geopandas as gpd

station = gpd.read_file(r"C:\\Users\\tengd\\OneDrive - The Hong Kong Polytechnic University\\Student Assistant\\EVCharger\\ArcGISProcess\\EVCharger.gdb",layer="OCM_Station")
charging = gpd.read_file(r"C:\\Users\\tengd\\OneDrive - The Hong Kong Polytechnic University\\Student Assistant\\EVCharger\\ArcGISProcess\\EVCharger.gdb",layer="OCM_Charging")

station.drop(station.loc[station["City"] == "X"].index, inplace=True)
station.set_index("UUID", inplace=True)

In [38]:
pointsNum = charging.groupby("UUID")["Correct_Quantity"].sum().reset_index()
pointsNum.set_index("UUID", inplace=True)
station = station.join(pointsNum, how="left")
station["ChargingNum"] = station[["Correct_Quantity", "correct_NumberOfPoints"]].max(axis=1).astype(int)

In [39]:
chargingNum = station.groupby(["country", "City"])["ChargingNum"].sum().reset_index()
chargingNum.set_index(["country", "City"], inplace=True)
stationNum = station.groupby(["country", "City"]).size().reset_index(name="StationNum")
stationNum.set_index(["country", "City"], inplace=True)

In [43]:
all = stationNum.join(chargingNum, how="left")
all.to_csv("statistic_clean.csv", encoding="utf-8")
all.groupby("country").sum().to_csv("statistic_clean_country.csv", encoding="utf-8")